# Quick, Draw!

## Some Useful Links
### Main
- Main Page: [QuickDraw](https://quickdraw.withgoogle.com)
- QuickDraw dataset repository, with many links to projects using it:
[here on github](https://github.com/googlecreativelab/quickdraw-dataset)

### Additional
- Demo drawings: [here](https://googlecreativelab.github.io/quickdraw-component/demo/#)
- closest to Zevi one can possibly get: [here](http://xinyue.de/scribbling-speech.html)
- in browser with [demo](https://zaidalyafeai.github.io/sketcher/) / [article](https://medium.com/tensorflow/train-on-google-colab-and-run-on-the-browser-a-case-study-8a45f9b1474e) / [notebook](https://colab.research.google.com/github/zaidalyafeai/zaidalyafeai.github.io/blob/master/sketcher/Sketcher.ipynb) / [code](https://github.com/zaidalyafeai/zaidalyafeai.github.io/tree/master/sketcher)
- python api: [here](https://quickdraw.readthedocs.io/en/latest/)
- sketch RNN blogpost: [here](https://ai.googleblog.com/2017/04/teaching-machines-to-draw.html)
- pix2pix for browser: [here](https://github.com/zaidalyafeai/zaidalyafeai.github.io/tree/master/pix2pix)
- magenta autocomplete: [here](https://magenta.tensorflow.org/sketch-rnn-demo)
- sketch RNN JS: [here](https://tensorflow.github.io/magenta-js/sketch/)
- TSNE mapping: [here](https://medium.com/@enjalot/machine-learning-for-visualization-927a9dff1cab)
- great tsne maps: [here](http://formafluens.io/client/iconolap.html)

## Helper Functions

### First

In [0]:
# Copyright 2017 Google Inc.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
# https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import struct
from struct import unpack


def unpack_drawing(file_handle):
    key_id, = unpack('Q', file_handle.read(8))
    countrycode, = unpack('2s', file_handle.read(2))
    recognized, = unpack('b', file_handle.read(1))
    timestamp, = unpack('I', file_handle.read(4))
    n_strokes, = unpack('H', file_handle.read(2))
    image = []
    for i in range(n_strokes):
        n_points, = unpack('H', file_handle.read(2))
        fmt = str(n_points) + 'B'
        x = unpack(fmt, file_handle.read(n_points))
        y = unpack(fmt, file_handle.read(n_points))
        image.append((x, y))

    return {
        'key_id': key_id,
        'countrycode': countrycode,
        'recognized': recognized,
        'timestamp': timestamp,
        'image': image
    }


def unpack_drawings(filename):
    with open(filename, 'rb') as f:
        while True:
            try:
                yield unpack_drawing(f)
            except struct.error:
                break

In [0]:
def get_drawings(category):
    drawings = []
    for drawing in unpack_drawings(category+'.bin'):
        drawings.append(drawing)
    return drawings

In [0]:
import matplotlib.pyplot as plt

### Second

In [4]:
!git clone 'https://github.com/dbt-ethz/sketch_rnn_keras_mod.git'

Cloning into 'sketch_rnn_keras_mod'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 18 (delta 1), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [5]:
from sketch_rnn_keras_mod.utils import *
#from sketch_rnn_keras_mod.seq2seqVAE_train import *
#import sketch_rnn_keras_mod.seq2seqVAE as sketch_rnn_model
from sketch_rnn_keras_mod.seq2seqVAE import *

# import the required libraries
import numpy as np
import time
import random
#import cPickle
import codecs
import collections
import os
import math
import json
import tensorflow as tf
from six.moves import xrange

Using TensorFlow backend.


In [0]:
# libraries required for visualisation:
from IPython.display import SVG, display
import PIL
from PIL import Image

# set numpy output to something sensible
np.set_printoptions(precision=8, edgeitems=6, linewidth=200, suppress=True)

In [7]:
!pip install -qU svgwrite

     |████████████████████████████████| 71kB 2.7MB/s 


In [0]:
import svgwrite

In [9]:
!wget 'http://storage.googleapis.com/quickdraw_dataset/sketchrnn/cat.npz'

--2019-05-10 09:53:51--  http://storage.googleapis.com/quickdraw_dataset/sketchrnn/cat.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14842947 (14M) [application/octet-stream]
Saving to: ‘cat.npz’

cat.npz             100%[===================>]  14.16M  80.4MB/s    in 0.2s    

2019-05-10 09:53:51 (80.4 MB/s) - ‘cat.npz’ saved [14842947/14842947]



In [10]:
!wget 'http://storage.googleapis.com/quickdraw_dataset/sketchrnn/guitar.npz'

--2019-05-10 09:53:53--  http://storage.googleapis.com/quickdraw_dataset/sketchrnn/guitar.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15700945 (15M) [application/octet-stream]
Saving to: ‘guitar.npz’

guitar.npz          100%[===================>]  14.97M  66.8MB/s    in 0.2s    

2019-05-10 09:53:54 (66.8 MB/s) - ‘guitar.npz’ saved [15700945/15700945]



In [0]:
# Function for encoding input and retrieving latent vector
def encode(input_strokes, draw=False):
    strokes = to_big_strokes(input_strokes, max_len=model_params['max_seq_len']-1).tolist()
    strokes.insert(0, [0, 0, 1, 0, 0])
    seq_len = [len(input_strokes)]
    #if draw:
    #    draw_strokes(to_normal_strokes(np.array(strokes)))
    strokes = np.expand_dims(strokes, axis=0)
    return seq2seq.sample_models['encoder_model'].predict(strokes)

In [0]:
# Function for decoding a latent space factor into a sketch
def decode(z_input=None, draw_mode=False, temperature=0.1, factor=0.2):
    z = None
    if z_input is not None:
        z = z_input
    sample_strokes, m = sample(seq2seq, seq_len=model_params.max_seq_len, temperature=temperature, z=z)
    strokes = to_normal_strokes(sample_strokes)
    #if draw_mode:
    #    draw_strokes(strokes, factor)
    return strokes

In [0]:
def interpolate_z(z_0, z_1, num=10, draw=False):
    z_list = [] # interpolate spherically between z0 and z1
    N = num
    for t in np.linspace(0, 1, N):
        z_list.append(slerp(z_0, z_1, t))
    # for every latent vector in z_list, sample a vector image
    reconstructions = []
    for i in range(N):
        reconstructions.append([decode(np.expand_dims(z_list[i],axis=0), draw_mode=False, temperature=0.01), [0, i]])
    
    #stroke_grid = make_grid_svg(reconstructions)
    #if draw:
    #    draw_strokes(stroke_grid)
    return reconstructions

### Third

In [14]:
weights = 'sketch_rnn_keras_mod/model/weights.hdf5' # checkpoint path
seq2seq = Seq2seqModel(model_params)  # build model
seq2seq.load_trained_weights(weights) # load checkpoint
seq2seq.make_sampling_models()  # build sub models that are used to infuse inputs and probe values of intermediate layers

NameError: ignored

## Download Dataset

Full list of available categories can be found [here](https://github.com/googlecreativelab/quickdraw-dataset/blob/master/categories.txt).

In [0]:
!wget 'http://storage.googleapis.com/quickdraw_dataset/full/binary/cat.bin'

## Visualise

In [0]:
cats = get_drawings('tiger')

In [0]:
len(cats)

In [0]:
plt.figure(figsize=(16,16))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.axis('off')
    mycat = cats[100+i]['image']
    for s in mycat:
        ys = s[1]
        ys = [-y for y in ys]
        plt.plot(s[0],ys)

In [0]:
plt.figure(figsize=(16,16))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.axis('off')
    mycat = cats[i]['image']
    #plt.title(cats[i]['countrycode'])
    col = 'ff0088'
    #if cats[i]['recognized']==0:
    #    col = 'r'
    for s in mycat:
        ys = s[1]
        ys = [-y for y in ys]
        plt.plot(s[0],ys,col)

In [0]:
data_dir = './'
with open('sketch_rnn_keras_mod/model/model_config.json','r') as f:
    model_params = json.load(f)
model_params = DotDict(model_params)

In [0]:
model_params

In [0]:
[train_set, valid_set, test_set, hps_model] = load_dataset(data_dir, model_params)

In [0]:
mysketch = test_set.random_sample()
myabssk = strokes_to_lines(mysketch)
plt.axis('equal')
plt.axis('off')
for s in myabssk:
    xs = [v[0] for v in s]
    ys = [-v[1] for v in s]
    plt.plot(xs,ys,'#FF0088')

In [0]:
z_cat = encode(mysketch)

In [0]:
z_cat

In [0]:
decoded_cat = decode(z_cat,temperature=0.001)

In [0]:
myabssk = strokes_to_lines(decoded_cat)
plt.axis('equal')
plt.axis('off')
for s in myabssk:
    xs = [v[0] for v in s]
    ys = [-v[1] for v in s]
    plt.plot(xs,ys,'#FF0088')

In [0]:
mysketch = test_set.strokes[8]
myabssk = strokes_to_lines(mysketch)
plt.axis('equal')
plt.axis('off')
for s in myabssk:
    xs = [v[0] for v in s]
    ys = [-v[1] for v in s]
    plt.plot(xs,ys,'#FF0088')

In [0]:
mysketch = test_set.strokes[1008]
myabssk = strokes_to_lines(mysketch)
plt.axis('equal')
plt.axis('off')
for s in myabssk:
    xs = [v[0] for v in s]
    ys = [-v[1] for v in s]
    plt.plot(xs,ys,'#FF0088')

In [0]:
z0 = encode(test_set.strokes[8])
z1 = encode(test_set.strokes[1008])

In [0]:
z0 = np.squeeze(z0)
z1 = np.squeeze(z1)

In [0]:
num = 15
blends = interpolate_z(z0,z1,num=num)

In [0]:
plt.figure(figsize=(15,1))
plt.axis('equal')
for i in range(num):
    step = strokes_to_lines(blends[i][0])
    plt.subplot(1,num,i+1)
    plt.axis('off')
    for s in step:
        xs = [v[0] for v in s]
        ys = [-v[1] for v in s]
        plt.plot(xs,ys,'k')

In [0]:
#z1.shape
invcat = decode(-z1,temperature=0.01)

In [0]:
z1.shape

In [0]:
myabssk = strokes_to_lines(invcat)
plt.axis('equal')
plt.axis('off')
for s in myabssk:
    xs = [v[0] for v in s]
    ys = [-v[1] for v in s]
    plt.plot(xs,ys,'#FF0088')

In [0]:
from sklearn import manifold

In [0]:
X = np.empty((1000,128))
for i in range(1000):
    s = test_set.random_sample()
    X[i,:] = encode(s)

In [0]:
tsne = manifold.TSNE(n_components=2, init='pca', perplexity=80)
X_tsne = tsne.fit_transform(X)

In [0]:
plt.figure(figsize=(10,10))
plt.scatter(*X_tsne.T)

In [0]:
X_tsne.shape

In [0]:
indices = []
for i in range(1000):
    if abs(X_tsne[i,0])<0.2 and abs(X_tsne[i,1])<0.2:
        indices.append(i)

In [0]:
len(indices)

In [0]:
indices

In [0]:
plt.figure(figsize=(15,15))
plt.axis('equal')
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.axis('off')
    idx = indices[i]
    #print(idx)
    decoded = decode(np.array(X[idx]).reshape(1,128))
    decstrokes = strokes_to_lines(decoded)
    for s in decstrokes:
        xs = [v[0] for v in s]
        ys = [-v[1] for v in s]
        plt.plot(xs,ys,'k')

In [0]:
X[22].shape